In [1]:
from pydantic import BaseModel, Field
from typing import List
import guidance
import os


In [2]:
# # vllm hosted model using LiteLLM

# litellm_desc = {
#     "model_name": "Qwen/Qwen3-30B-A3B-Instruct-2507",
#     "litellm_params": {  # params for litellm completion/embedding call
#         "model": "hosted_vllm/Qwen/Qwen3-30B-A3B-Instruct-2507",
#         "api_key": os.environ.get("VLLM_API_KEY", "NO_KEY"), # set your vLLM API key if needed
#         "api_base": "http://localhost:8000/v1", # change to your vLLM API base URL
#     },
# }
# base_lm = guidance.models.experimental.LiteLLM(model_description=litellm_desc, echo=True)

In [2]:
base_lm = guidance.models.experimental.SglangModel(model="Qwen/Qwen3-30B-A3B-Instruct-2507", base_url="http://127.0.0.1:30000/v1" ,echo=True, api_key = os.environ.get("OPENAI_API_KEY", "NO_KEY"))

In [3]:
# def run_gen_test(lm):
#     with guidance.user():
#         lm += "What is the capital of France? and its population?"
#         lm += "Format your answer as follows: Capital: <capital>, Population: <population>"

#     with guidance.assistant():
#         lm += guidance.gen(max_tokens=1024, temperature=0.7, name="answer")
#         print(lm["answer"])

# run_gen_test(base_lm)

In [3]:
from pydantic import BaseModel, Field
from typing import List
import guidance
from guidance import models, system, user, assistant, json as gen_json, select
import json
from enum import Enum


class FirstPrinciples(BaseModel):
    absolute_truths: List[str] = Field(
        ...,
        description="Look at the problem statement and list the things that you know that are absolute truths."
    )


class GroundUpReasoning(BaseModel):
    """Model for the LLM-generated reasoning plan."""
    reason: str = Field(
        ...,
        description="Based on the known truths reason in a concise sharp manner from ground up to address the problem."
    )

class Reflection(BaseModel):
    """Model for an individual reasoning step execution."""
    evaluation: str = Field(
        ...,
        description="Reflect on your reasoning and identify gaps / uncertainties / mistakes."
    )

class ReasoningConclusion(BaseModel):
    """Model for the final reasoning conclusion."""
    conclusion: str = Field(
        ...,
        description="Address the identified gaps in the previous step if any and deduce the final conclusion."
    )

def dynamic_final_answer_schema(Options):
    class FinalAnswer(BaseModel):
        answer: Options = Field(..., description="Final answer.")

    return FinalAnswer


@guidance()
def _first_principles(llm):
    with assistant():
        llm += gen_json(
            name="first_principles",
            schema=FirstPrinciples,
            max_tokens=5000
        )
    return llm

@guidance()
def _generate_groundup_reasoning(llm):
    """
    Generate a custom reasoning plan tailored to the query.
    """
    with assistant():
        llm += gen_json(
            name="groundup_reasoning",
            schema=GroundUpReasoning,
            max_tokens=5000  # Reduced to encourage conciseness
        )
    return llm

@guidance()
def _reflection(llm):
    with assistant():
        llm += gen_json(
            name="reflection",
            schema=Reflection,
            max_tokens=5000
        )
    return llm
    

@guidance()
def _reasoning_conclusion(llm):
    with user():
        llm += """Based on the information, make logical deductions."""
    
    with assistant():
        llm += gen_json(
            name="reasoning_conclusion",
            schema=ReasoningConclusion,
            max_tokens=5000
        )

    return llm

@guidance
def dynamic_structured_reasoning(llm, query, output_choices):
    Options = Enum("Choices", {choice: choice for choice in output_choices})
    final_answer_schema = dynamic_final_answer_schema(Options)

    with system():
        llm += """You are an expert in problem solving. For any query, reason from first principles to solve the query. 
        Structure all outputs as JSON. Be very brief in your steps, tending towards concise but complete responses.
        """

    with user():
        llm += f"""Query: {query}
        Look at the problem statement and list the things that you know that are absolute truths -
        """

    llm += _first_principles()

    with user():
        llm += f"""
        Based on the known truths reason from ground up to address the problem, be extremely brief and to the point in your reasoning -
        """

    llm += _generate_groundup_reasoning()

    with user():
        llm += f"""
        Reflect to identify gaps / uncertainties / mistakes in reasoning process -
        """

    llm += _reflection()

    with user():
        llm += f"""
        Make logical deductions based on the information you have to arrive at a conclusion -
        """

    llm += _reasoning_conclusion()

    with user():
        llm += f"""
        Your final answer -
        """

    with assistant():
        llm += gen_json(
            name="answer",
            schema=final_answer_schema,
            max_tokens=1000)

    return llm

In [4]:
def generate(query, output_choices, lm = None):
    lm = lm or base_lm
    result_lm = lm + dynamic_structured_reasoning(
                    query=query,
                    output_choices = output_choices
                )

    answer = json.loads(result_lm.get("answer"))['answer']
    
    groundup_reasoning = json.loads(result_lm.get("groundup_reasoning"))['reason']
    first_principles = json.loads(result_lm.get("first_principles"))['absolute_truths']
    reflection = json.loads(result_lm.get("reflection"))['evaluation']   
    reasoning_conclusion = json.loads(result_lm.get("reasoning_conclusion"))['conclusion']
    return first_principles, groundup_reasoning, reflection, reasoning_conclusion, answer


In [5]:
from datasets import load_dataset

# Load the BBH dataset from Hugging Face
dataset = load_dataset("WildEval/ZebraLogic", 'mc_mode')



In [6]:
from json import JSONDecodeError


def evaluate_reasoning(row):
    """
    Placeholder function to evaluate your reasoning system on a single row.
    Replace with your dynamic_structured_reasoning function.
    """
    
    # from guidance.models.experimental import SglangModel
    # lm = SglangModel(
    #     model="Qwen/Qwen3-30B-A3B-Instruct-2507",
    #     base_url="http://127.0.0.1:30000/v1",
    #     echo=True,
    #     api_key=os.environ.get("OPENAI_API_KEY", "NO_KEY"),
    # )
    puzzle = row['puzzle']
    question = row['question']
    query = puzzle + '\n\n' + question
    choices = row['choices']
    target = row['answer']
    print(row['id'])
    # Your reasoning system should return a predicted answer
    try:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = generate(query, choices)
    except JSONDecodeError as e:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = ["JSONDecodeError"]*5
    return {
        'correct': pred == target,
        'first_principles': first_principles,
        'groundup_reasoning': groundup_reasoning,
        'reflection': reflection,
        'reasoning_conclusion': reasoning_conclusion,
        'predicted_answer': pred, 
    }


In [7]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import pandas as pd

# # df is already df.iloc[600:650]  (keeps original index)
# def run_parallel_preserve_index(df, workers=16):
#     idx_to_row = df.to_dict(orient="index")  # {index: rowdict}
#     results = {}

#     with ThreadPoolExecutor(max_workers=workers) as ex:
#         futs = {ex.submit(evaluate_reasoning, row): idx
#                 for idx, row in idx_to_row.items()}
#         for fut in as_completed(futs):
#             idx = futs[fut]
#             results[idx] = fut.result()

#     res_df = pd.DataFrame.from_dict(results, orient="index")
#     # ensure order matches df (and keep original index)
#     res_df = res_df.reindex(df.index)
#     return df.join(res_df)

# df = pd.DataFrame(dataset['test'])
# # df.head()
# # CURRENTLY FINISHED - [:600]

# df = df.iloc[600:605]
# df = run_parallel_preserve_index(df, workers=16)


In [8]:
# import pandas as pd
# df = pd.DataFrame(dataset['test'])
# len(df)

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset['test'])
# df.head()
# CURRENTLY FINISHED - [:1250]

df = df.iloc[1250:1350]

res = df.apply(lambda r: pd.Series(evaluate_reasoning(r)), axis=1)
df = pd.concat([df, res], axis=1)

lgp-test-6x4-11#mc-2


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

lgp-test-5x4-18#mc-9
lgp-test-6x5-39#mc-12
lgp-test-6x6-33#mc-26
lgp-test-3x6-16#mc-15
lgp-test-3x4-15#mc-10
lgp-test-4x2-6#mc-1
lgp-test-6x4-27#mc-16
lgp-test-5x5-11#mc-3
lgp-test-6x6-39#mc-32
lgp-test-5x2-7#mc-8
lgp-test-2x4-1#mc-5
lgp-test-6x5-14#mc-22
lgp-test-4x4-6#mc-8
lgp-test-3x6-36#mc-3
lgp-test-5x6-12#mc-23
lgp-test-4x4-12#mc-13
lgp-test-6x2-2#mc-11
lgp-test-5x6-6#mc-5
lgp-test-5x5-16#mc-17
lgp-test-5x5-17#mc-20
lgp-test-4x5-37#mc-9
lgp-test-2x6-34#mc-11
lgp-test-4x3-6#mc-1
lgp-test-2x3-13#mc-3
lgp-test-4x6-12#mc-2
lgp-test-5x5-19#mc-2
lgp-test-5x6-37#mc-21
lgp-test-2x6-0#mc-7
lgp-test-5x6-29#mc-19
lgp-test-6x4-10#mc-5
lgp-test-4x5-12#mc-19
lgp-test-6x6-30#mc-26
lgp-test-5x6-11#mc-4
lgp-test-6x6-18#mc-27
lgp-test-3x5-5#mc-5
lgp-test-3x5-39#mc-7
lgp-test-2x4-5#mc-4
lgp-test-5x5-14#mc-5
lgp-test-2x2-13#mc-1
lgp-test-2x3-22#mc-3
lgp-test-4x6-5#mc-19
lgp-test-6x6-7#mc-6
lgp-test-5x6-39#mc-29
lgp-test-4x4-28#mc-15
lgp-test-4x5-22#mc-9
lgp-test-2x4-11#mc-5
lgp-test-4x5-0#mc-3
lgp-t

In [16]:
df

,id,puzzle,question,choices,answer,created_at,correct,first_principles,groundup_reasoning,reflection,reasoning_conclusion,predicted_answer
1200,lgp-test-6x4-11#mc-2,"There are 6 houses, numbered 1 to 6 from left ...",What is MusicGenre of the person who lives in ...,"[jazz, classical, rock, country, pop, hip hop]",country,2024-07-03T21:21:31.371153,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1201,lgp-test-5x4-18#mc-9,"There are 5 houses, numbered 1 to 5 from left ...",What is CarModel of the person who lives in Ho...,"[tesla model 3, honda civic, toyota camry, for...",toyota camry,2024-07-03T21:21:31.345335,True,[House 1 has mother's name: Holly (from clue 8...,"Eric owns BMW 3 Series, loves green, and is no...",The reasoning is logically sound and consisten...,The person in House 3 owns the Toyota Camry.,toyota camry
1202,lgp-test-6x5-39#mc-12,"There are 6 houses, numbered 1 to 6 from left ...",What is Drink of the person who lives in House 3?,"[water, coffee, root beer, milk, boba tea, tea]",water,2024-07-03T21:21:31.383988,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1203,lgp-test-6x6-33#mc-26,"There are 6 houses, numbered 1 to 6 from left ...",What is Nationality of the person who lives in...,"[norwegian, german, chinese, swede, dane, brit]",swede,2024-07-03T21:21:31.393476,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1204,lgp-test-3x6-16#mc-15,"There are 3 houses, numbered 1 to 3 from left ...",What is Education of the person who lives in H...,"[high school, associate, bachelor]",associate,2024-07-03T21:21:31.316253,True,[House 2 has the person with a bachelor's degr...,House 2 has bachelor's degree (Clue 3). High s...,All constraints were applied consistently. Key...,The person in House 3 has an associate degree.,associate
1205,lgp-test-3x4-15#mc-10,"There are 3 houses, numbered 1 to 3 from left ...",What is BookGenre of the person who lives in H...,"[science fiction, mystery, romance]",romance,2024-07-03T21:21:31.310113,True,"[Arnold is in House 3., The person who loves s...",Arnold is in House 3. Eric is the science fict...,Earlier contradiction arose from assuming teac...,The person in House 3 has the book genre 'roma...,romance
1206,lgp-test-4x2-6#mc-1,"There are 4 houses, numbered 1 to 4 from left ...",What is Cigar of the person who lives in House 1?,"[prince, blue master, dunhill, pall mall]",dunhill,2024-07-03T21:21:31.318738,True,"[House 1 is occupied by Alice (from clue 4)., ...",House 1 is Alice (clue 4). Prince not in House...,Reasoning is sound: all clues applied. Key ste...,The cigar in House 1 is Dunhill.,dunhill
1207,lgp-test-6x4-27#mc-16,"There are 6 houses, numbered 1 to 6 from left ...",What is Name of the person who lives in House 5?,"[Bob, Peter, Eric, Alice, Carol, Arnold]",Carol,2024-07-03T21:21:31.373709,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1208,lgp-test-5x5-11#mc-3,"There are 5 houses, numbered 1 to 5 from left ...",What is Smoothie of the person who lives in Ho...,"[lime, watermelon, dragonfruit, desert, cherry]",watermelon,2024-07-03T21:21:31.349625,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1209,lgp-test-6x6-39#mc-32,"There are 6 houses, numbered 1 to 6 from left ...",What is Hobby of the person who lives in House 6?,"[knitting, cooking, woodworking, painting, pho...",photography,2024-07-03T21:21:31.395163,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError


In [17]:
df['correct'].value_counts()

correct
True     33
False    17
Name: count, dtype: int64

In [18]:
path = './zebralogic_eval.csv'
df.to_csv(path, mode="a", index=False, header=not os.path.exists(path))

In [19]:
dub = pd.read_csv(path)


In [20]:
dub['correct'].value_counts()

correct
True     922
False    328
Name: count, dtype: int64